# The Associated Press and Life Expectancy

**Story:** [AP analysis: Unemployment, income affect life expectancy](https://www.apnews.com/66ac44186b6249709501f07a7eab36da)

**Author:** Nicky Forster, Associated Press

**Topics:** Census Data, Linear Regression

**Datasets**

* **R12221544_SL140.csv:** ACS 2015 5-year, tract level, from [Social Explorer](https://www.socialexplorer.com)
    - Table B23025: Employment Status
    - **R12221544.txt** is the data dictionary
* **R12221550_SL140.csv:** ACS 2015 5-year, tract level, from [Social Explorer](https://www.socialexplorer.com)
    - Table B23025: Employment Status
    - Table B06009: Educational Attainment
    - Table B03002: Race
    - Table B19013: Median income
    - Table C17002: Ratio of income to poverty level
    - **R12221550.txt** is the data dictionary
* **US_A.CSV:** life expectancy by census tract, from [USALEEP](https://www.cdc.gov/nchs/nvss/usaleep/usaleep.html)
    - **Record_Layout_CensusTract_Life_Expectancy.pdf** is data dictionary

# What's the story?

We're trying to figure out how the **life expectancy in a census tract** is related to other factors like unemployment, income, and others.

<p class="reading-options">
  <a class="btn" href="/ap-regression-unemployment/associated-press-and-life-expectancy">
    <i class="fa fa-sm fa-book"></i>
    Read online
  </a>
  <a class="btn" href="/ap-regression-unemployment/notebooks/Associated Press and life expectancy.ipynb">
    <i class="fa fa-sm fa-download"></i>
    Download notebook
  </a>
  <a class="btn" href="https://colab.research.google.com/github/littlecolumns/ds4j-notebooks/blob/master/ap-regression-unemployment/notebooks/Associated Press and life expectancy.ipynb" target="_new">
    <i class="fa fa-sm fa-laptop"></i>
    Interactive version
  </a>
</p>

### Prep work: Downloading necessary files
Before we get started, we need to download all of the data we'll be using.
* **R12221544_SL140.csv:** Employment status by census tract - ACS 2015 5-year, tract level, table B23025
* **R12221544.txt:** Data dictionary for R12221544_SL140.csv - Description of columns and tables
* **R12221550_SL140.csv:** Demographic data by census tract - ACS 2015 5-year, tract level, tables B23025, B06009, B03002, B19013, C17002
* **R12221550.txt:** Data dictionary for R12221550_SL140.csv - Description of columns and tables
* **US_A.CSV:** USALEEP national data - estimates of life expectancy at birth for most of the census tracts in the USA
* **Record_Layout_CensusTract_Life_Expectancy.pdf:** USALEEP data dictionary - None


In [ ]:
# Make data directory if it doesn't exist
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/ap-regression-unemployment/data/R12221544_SL140.csv.zip -P data
!unzip -n -d data data/R12221544_SL140.csv.zip
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/ap-regression-unemployment/data/R12221544.txt -P data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/ap-regression-unemployment/data/R12221550_SL140.csv.zip -P data
!unzip -n -d data data/R12221550_SL140.csv.zip
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/ap-regression-unemployment/data/R12221550.txt -P data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/ap-regression-unemployment/data/US_A.CSV -P data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/ap-regression-unemployment/data/Record_Layout_CensusTract_Life_Expectancy.pdf -P data

# PREPWORK BONUS!

Download the data yourself from Social Explorer and USALEEP (linked above) instead of relying on the data included.

## Imports

In [2]:
import pandas as pd

pd.set_option("display.max_columns", 100)

## Reading in our data

### Read in `USA_A.CSV`

Rename any columns with weird or not-understandable names as something more descriptive.

In [3]:
life_expec = pd.read_csv("data/US_A.CSV")
life_expec.columns = ['tract_id', 'STATE2KX','CNTY2KX', 'TRACT2KX', 'life_expectancy', 'life_expectancy_std_err', 'flag']
life_expec.head()

tract_id  STATE2KX  CNTY2KX  TRACT2KX  life_expectancy  \
0  1001020100         1        1     20100             73.1   
1  1001020200         1        1     20200             76.9   
2  1001020400         1        1     20400             75.4   
3  1001020500         1        1     20500             79.4   
4  1001020600         1        1     20600             73.1   

   life_expectancy_std_err  flag  
0                   2.2348     3  
1                   3.3453     3  
2                   1.0216     3  
3                   1.1768     1  
4                   1.5519     3

### Open `R12221544_SL140.csv`

You'll need to give an option to `pd.read_csv` to make sure it's read in successfully.

In [32]:
employment = pd.read_csv("data/R12221544_SL140.csv", encoding='latin-1')
employment.head()

Geo_FIPS           Geo_GEOID                                   Geo_NAME  \
0  1001020100  14000US01001020100  Census Tract 201, Autauga County, Alabama   
1  1001020200  14000US01001020200  Census Tract 202, Autauga County, Alabama   
2  1001020300  14000US01001020300  Census Tract 203, Autauga County, Alabama   
3  1001020400  14000US01001020400  Census Tract 204, Autauga County, Alabama   
4  1001020500  14000US01001020500  Census Tract 205, Autauga County, Alabama   

                                   Geo_QName Geo_STUSAB  Geo_SUMLEV  \
0  Census Tract 201, Autauga County, Alabama         al         140   
1  Census Tract 202, Autauga County, Alabama         al         140   
2  Census Tract 203, Autauga County, Alabama         al         140   
3  Census Tract 204, Autauga County, Alabama         al         140   
4  Census Tract 205, Autauga County, Alabama         al         140   

   Geo_GEOCOMP Geo_FILEID  Geo_LOGRECNO  Geo_US  Geo_REGION  Geo_DIVISION  \
0            0      ACSSF          1760     NaN         NaN           NaN   
1            0      ACSSF          1761     NaN         NaN           NaN   
2            0      ACSSF          1762     NaN         NaN           NaN   
3            0      ACSSF          1763     NaN         NaN           NaN   
4            0      ACSSF          1764     NaN         NaN           NaN   

   Geo_STATECE  Geo_STATE  Geo_COUNTY  Geo_COUSUB  Geo_PLACE  Geo_PLACESE  \
0          NaN          1           1         NaN        NaN          NaN   
1          NaN          1           1         NaN        NaN          NaN   
2          NaN          1           1         NaN        NaN          NaN   
3          NaN          1           1         NaN        NaN          NaN   
4          NaN          1           1         NaN        NaN          NaN   

   Geo_TRACT  Geo_BLKGRP  Geo_CONCIT  Geo_AIANHH  Geo_AIANHHFP  Geo_AIHHTLI  \
0      20100         NaN         NaN         NaN           NaN          NaN   
1      20200         NaN         NaN         NaN           NaN          NaN   
2      20300         NaN         NaN         NaN           NaN          NaN   
3      20400         NaN         NaN         NaN           NaN          NaN   
4      20500         NaN         NaN         NaN           NaN          NaN   

   Geo_AITSCE  Geo_AITS  Geo_ANRC  Geo_CBSA  Geo_CSA  Geo_METDIV  Geo_MACC  \
0         NaN       NaN       NaN       NaN      NaN         NaN       NaN   
1         NaN       NaN       NaN       NaN      NaN         NaN       NaN   
2         NaN       NaN       NaN       NaN      NaN         NaN       NaN   
3         NaN       NaN       NaN       NaN      NaN         NaN       NaN   
4         NaN       NaN       NaN       NaN      NaN         NaN       NaN   

   Geo_MEMI  Geo_NECTA  Geo_CNECTA  Geo_NECTADIV  Geo_UA  Geo_UACP  \
0       NaN        NaN         NaN           NaN     NaN       NaN   
1       NaN        NaN         NaN           NaN     NaN       NaN   
2       NaN        NaN         NaN           NaN     NaN       NaN   
3       NaN        NaN         NaN           NaN     NaN       NaN   
4       NaN        NaN         NaN           NaN     NaN       NaN   

   Geo_CDCURR  Geo_SLDU  Geo_SLDL  Geo_VTD  Geo_ZCTA3  Geo_ZCTA5  Geo_SUBMCD  \
0         NaN       NaN       NaN      NaN        NaN        NaN         NaN   
1         NaN       NaN       NaN      NaN        NaN        NaN         NaN   
2         NaN       NaN       NaN      NaN        NaN        NaN         NaN   
3         NaN       NaN       NaN      NaN        NaN        NaN         NaN   
4         NaN       NaN       NaN      NaN        NaN        NaN         NaN   

   Geo_SDELM  Geo_SDSEC  Geo_SDUNI  Geo_UR  Geo_PCI  Geo_TAZ  Geo_UGA  \
0        NaN        NaN        NaN     NaN      NaN      NaN      NaN   
1        NaN        NaN        NaN     NaN      NaN      NaN      NaN   
2        NaN        NaN        NaN     NaN      NaN      NaN      NaN   
3        NaN        NaN        Na

#### Filter out any columns we aren't interestd in

In [33]:
employment = employment[['Geo_FIPS', 'ACS15_5yr_B23025001', 'ACS15_5yr_B23025002', \
                         'ACS15_5yr_B23025003', 'ACS15_5yr_B23025004', 'ACS15_5yr_B23025005', \
                         'ACS15_5yr_B23025006', 'ACS15_5yr_B23025007']].copy()
employment.head()

Geo_FIPS  ACS15_5yr_B23025001  ACS15_5yr_B23025002  ACS15_5yr_B23025003  \
0  1001020100                 1554                  997                  997   
1  1001020200                 1731                  884                  869   
2  1001020300                 2462                 1472                 1464   
3  1001020400                 3424                 2013                 1998   
4  1001020500                 8198                 5461                 5258   

   ACS15_5yr_B23025004  ACS15_5yr_B23025005  ACS15_5yr_B23025006  \
0                  943                   54                    0   
1                  753                  116                   15   
2                 1373                   91                    8   
3                 1782                  216                   15   
4                 5037                  221                  203   

   ACS15_5yr_B23025007  
0                  557  
1                  847  
2                  990  
3                 1411  
4                 2737

#### Create a new column for percent unemployment

We'll be using the total population in the census tract as the baseline for employment.

In [34]:
employment['pct_unemployment'] = employment['ACS15_5yr_B23025005'] / employment['ACS15_5yr_B23025001'] * 100
employment.head()

Geo_FIPS  ACS15_5yr_B23025001  ACS15_5yr_B23025002  ACS15_5yr_B23025003  \
0  1001020100                 1554                  997                  997   
1  1001020200                 1731                  884                  869   
2  1001020300                 2462                 1472                 1464   
3  1001020400                 3424                 2013                 1998   
4  1001020500                 8198                 5461                 5258   

   ACS15_5yr_B23025004  ACS15_5yr_B23025005  ACS15_5yr_B23025006  \
0                  943                   54                    0   
1                  753                  116                   15   
2                 1373                   91                    8   
3                 1782                  216                   15   
4                 5037                  221                  203   

   ACS15_5yr_B23025007  pct_unemployment  
0                  557          3.474903  
1                  847          6.701329  
2                  990          3.696182  
3                 1411          6.308411  
4                 2737          2.695779

**What do we mean by "percent?"** There' actually a difference between using `0-1.0` compared to using the `0-100` integer version! After you've completed the notebook with one, try changing it to the other and re-running the rest of the notebook.

## Merging the data

Merge the dataframes together based on their census tract.

In [35]:
merged = life_expec.merge(employment, left_on='tract_id', right_on='Geo_FIPS')
merged.head()

tract_id  STATE2KX  CNTY2KX  TRACT2KX  life_expectancy  \
0  1001020100         1        1     20100             73.1   
1  1001020200         1        1     20200             76.9   
2  1001020400         1        1     20400             75.4   
3  1001020500         1        1     20500             79.4   
4  1001020600         1        1     20600             73.1   

   life_expectancy_std_err  flag    Geo_FIPS  ACS15_5yr_B23025001  \
0                   2.2348     3  1001020100                 1554   
1                   3.3453     3  1001020200                 1731   
2                   1.0216     3  1001020400                 3424   
3                   1.1768     1  1001020500                 8198   
4                   1.5519     3  1001020600                 2855   

   ACS15_5yr_B23025002  ACS15_5yr_B23025003  ACS15_5yr_B23025004  \
0                  997                  997                  943   
1                  884                  869                  753   
2                 2013                 1998                 1782   
3                 5461                 5258                 5037   
4                 1802                 1750                 1560   

   ACS15_5yr_B23025005  ACS15_5yr_B23025006  ACS15_5yr_B23025007  \
0                   54                    0                  557   
1                  116                   15                  847   
2                  216                   15                 1411   
3                  221                  203                 2737   
4                  190                   52                 1053   

   pct_unemployment  
0          3.474903  
1          6.701329  
2          6.308411  
3          2.695779  
4          6.654991

## Running the regression

Using the `statsmodels` package, run a linear regression to find the coefficient relating unemployment and life expectancy.

In [41]:
import statsmodels.api as sm

X = merged[['pct_unemployment']]
X = sm.add_constant(X)
y = merged.life_expectancy

model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        life_expectancy   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.169
Method:                 Least Squares   F-statistic:                 1.336e+04
Date:                Tue, 10 Sep 2019   Prob (F-statistic):               0.00
Time:                        12:33:36   Log-Likelihood:            -1.7810e+05
No. Observations:               65662   AIC:                         3.562e+05
Df Residuals:                   65660   BIC:                         3.562e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               81.1377      0.028   2856.410      0.000      81.082      81.193
pct_unemployment    -0.5214      0.005   -115.595      0.000      -0.530      -0.513
==============================================================================
Omnibus:                      616.108   Durbin-Watson:                   1.117
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              807.895
Skew:                          -0.146   Prob(JB):                    3.70e-176
Kurtosis:                       3.459   Cond. No.                         12.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [65]:
import statsmodels.formula.api as smf

model = smf.ols(formula='life_expectancy ~ pct_unemployment', data=merged)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        life_expectancy   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.169
Method:                 Least Squares   F-statistic:                 1.336e+04
Date:                Tue, 10 Sep 2019   Prob (F-statistic):               0.00
Time:                        13:27:46   Log-Likelihood:            -1.7810e+05
No. Observations:               65662   AIC:                         3.562e+05
Df Residuals:                   65660   BIC:                         3.562e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           81.1377      0.028   2856.410      0.000      81.082      81.193
pct_unemployment    -0.5214      0.005   -115.595      0.000      -0.530      -0.513
==============================================================================
Omnibus:                      616.108   Durbin-Watson:                   1.117
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              807.895
Skew:                          -0.146   Prob(JB):                    3.70e-176
Kurtosis:                       3.459   Cond. No.                         12.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Translate that into the form **"every X percentage point change in unemployment translates to a Y change in life expectancy"**

In [39]:
# Every 1 point change in unemployment translates to a half-year decrease in life expectancy

## Bringing more columns into the mix

Only dealing with unemployment seems kind of narrow-minded, let's expand our reach a bit.

### Read in `R12221550_SL140.csv`

It's also from the Census, and has many, many, many more columns available to you compared to the list dataset.

In [44]:
census = pd.read_csv("data/R12221550_SL140.csv", encoding='latin-1')
census.head()

Geo_FIPS           Geo_GEOID                                   Geo_NAME  \
0  1001020100  14000US01001020100  Census Tract 201, Autauga County, Alabama   
1  1001020200  14000US01001020200  Census Tract 202, Autauga County, Alabama   
2  1001020300  14000US01001020300  Census Tract 203, Autauga County, Alabama   
3  1001020400  14000US01001020400  Census Tract 204, Autauga County, Alabama   
4  1001020500  14000US01001020500  Census Tract 205, Autauga County, Alabama   

                                   Geo_QName Geo_STUSAB  Geo_SUMLEV  \
0  Census Tract 201, Autauga County, Alabama         al         140   
1  Census Tract 202, Autauga County, Alabama         al         140   
2  Census Tract 203, Autauga County, Alabama         al         140   
3  Census Tract 204, Autauga County, Alabama         al         140   
4  Census Tract 205, Autauga County, Alabama         al         140   

   Geo_GEOCOMP Geo_FILEID  Geo_LOGRECNO  Geo_US  Geo_REGION  Geo_DIVISION  \
0            0      ACSSF          1760     NaN         NaN           NaN   
1            0      ACSSF          1761     NaN         NaN           NaN   
2            0      ACSSF          1762     NaN         NaN           NaN   
3            0      ACSSF          1763     NaN         NaN           NaN   
4            0      ACSSF          1764     NaN         NaN           NaN   

   Geo_STATECE  Geo_STATE  Geo_COUNTY  Geo_COUSUB  Geo_PLACE  Geo_PLACESE  \
0          NaN          1           1         NaN        NaN          NaN   
1          NaN          1           1         NaN        NaN          NaN   
2          NaN          1           1         NaN        NaN          NaN   
3          NaN          1           1         NaN        NaN          NaN   
4          NaN          1           1         NaN        NaN          NaN   

   Geo_TRACT  Geo_BLKGRP  Geo_CONCIT  Geo_AIANHH  Geo_AIANHHFP  Geo_AIHHTLI  \
0      20100         NaN         NaN         NaN           NaN          NaN   
1      20200         NaN         NaN         NaN           NaN          NaN   
2      20300         NaN         NaN         NaN           NaN          NaN   
3      20400         NaN         NaN         NaN           NaN          NaN   
4      20500         NaN         NaN         NaN           NaN          NaN   

   Geo_AITSCE  Geo_AITS  Geo_ANRC  Geo_CBSA  Geo_CSA  Geo_METDIV  Geo_MACC  \
0         NaN       NaN       NaN       NaN      NaN         NaN       NaN   
1         NaN       NaN       NaN       NaN      NaN         NaN       NaN   
2         NaN       NaN       NaN       NaN      NaN         NaN       NaN   
3         NaN       NaN       NaN       NaN      NaN         NaN       NaN   
4         NaN       NaN       NaN       NaN      NaN         NaN       NaN   

   Geo_MEMI  Geo_NECTA  Geo_CNECTA  Geo_NECTADIV  Geo_UA  Geo_UACP  \
0       NaN        NaN         NaN           NaN     NaN       NaN   
1       NaN        NaN         NaN           NaN     NaN       NaN   
2       NaN        NaN         NaN           NaN     NaN       NaN   
3       NaN        NaN         NaN           NaN     NaN       NaN   
4       NaN        NaN         NaN           NaN     NaN       NaN   

   Geo_CDCURR  Geo_SLDU  Geo_SLDL  Geo_VTD  Geo_ZCTA3  Geo_ZCTA5  Geo_SUBMCD  \
0         NaN       NaN       NaN      NaN        NaN        NaN         NaN   
1         NaN       NaN       NaN      NaN        NaN        NaN         NaN   
2         NaN       NaN       NaN      NaN        NaN        NaN         NaN   
3         NaN       NaN       NaN      NaN        NaN        NaN         NaN   
4         NaN       NaN       NaN      NaN        NaN        NaN         NaN   

   Geo_SDELM  Geo_SDSEC  Geo_SDUNI  Geo_UR  Geo_PCI  Geo_TAZ  ...  \
0        NaN        NaN        NaN     NaN      NaN      NaN  ...   
1        NaN        NaN        NaN     NaN      NaN      NaN  ...   
2        NaN        NaN        NaN     NaN      NaN      NaN  ...   
3        NaN        NaN        NaN     NaN      N

Using this census data, create a new dataframe that includes the following columns:

* Percent unemployed
* Percents Black, White, and Hispanic
* Median Income (in increments of 10,000 dollars)
* Percent of the population with less than a high school education
* Percent of the population between 1-1.5x the poverty line

If you have to many any editorial decisions about which columns you choose or how you do your math, please explain them.

In [55]:
features = pd.DataFrame({
    'Geo_FIPS': census.Geo_FIPS,
    'pct_black': census.ACS15_5yr_B03002004 / census.ACS15_5yr_B03002001 * 100,
    'pct_white': census.ACS15_5yr_B03002003 / census.ACS15_5yr_B03002001 * 100,
    'pct_hispanic': census.ACS15_5yr_B03002012 / census.ACS15_5yr_B03002001 * 100,
    'pct_less_than_hs': census.ACS15_5yr_B06009002 / census.ACS15_5yr_B06009001 * 100,
    'pct_1_15_poverty': (census.ACS15_5yr_C17002004 + census.ACS15_5yr_C17002005) / census.ACS15_5yr_C17002001 * 100,
    'income_10k': census.ACS15_5yr_B19013001 / 10000,
    'pct_unemployment': census.ACS15_5yr_B23025005 / census.ACS15_5yr_B23025001 * 100
})
features.head()

Geo_FIPS  pct_black  pct_white  pct_hispanic  pct_less_than_hs  \
0  1001020100   7.700205  87.422998      0.872690         14.802896   
1  1001020200  53.293135  40.445269      0.788497         25.483178   
2  1001020300  18.564690  74.528302      0.000000         10.655738   
3  1001020400   3.662672  82.794483     10.490617         11.693687   
4  1001020500  24.844374  68.456750      0.743287          4.445082   

   pct_1_15_poverty  income_10k  pct_unemployment  
0          9.342916      6.1838          3.474903  
1         14.624307      3.2303          6.701329  
2         11.960916      4.4922          3.696182  
3          7.302736      5.4329          6.308411  
4         12.496450      5.1965          2.695779

### Join your datasets

Combine your life expectancy dataset with this census dataset to create a new dataframe.

In [56]:
features = features.merge(life_expec, left_on='Geo_FIPS', right_on='tract_id')
features.head()

Geo_FIPS  pct_black  pct_white  pct_hispanic  pct_less_than_hs  \
0  1001020100   7.700205  87.422998      0.872690         14.802896   
1  1001020200  53.293135  40.445269      0.788497         25.483178   
2  1001020400   3.662672  82.794483     10.490617         11.693687   
3  1001020500  24.844374  68.456750      0.743287          4.445082   
4  1001020600  11.918982  72.916126     13.061542         17.487267   

   pct_1_15_poverty  income_10k  pct_unemployment    tract_id  STATE2KX  \
0          9.342916      6.1838          3.474903  1001020100         1   
1         14.624307      3.2303          6.701329  1001020200         1   
2          7.302736      5.4329          6.308411  1001020400         1   
3         12.496450      5.1965          2.695779  1001020500         1   
4         10.854324      6.3092          6.654991  1001020600         1   

   CNTY2KX  TRACT2KX  life_expectancy  life_expectancy_std_err  flag  
0        1     20100             73.1                   2.2348     3  
1        1     20200             76.9                   3.3453     3  
2        1     20400             75.4                   1.0216     3  
3        1     20500             79.4                   1.1768     1  
4        1     20600             73.1                   1.5519     3

#### Remove rows that are missing data

In [57]:
features.shape

(65662, 15)

In [58]:
features = features.dropna()
features.shape

(65656, 15)

## Running your multivariate regression

Using the `statsmodels` package and this new dataframe, run a multivariate linear regression to find the coefficient relating your columns and life expectancy.

In [59]:
import statsmodels.api as sm

X = features[['pct_black', 'pct_white', 'pct_hispanic', 'pct_less_than_hs', 'pct_1_15_poverty', 'income_10k', 'pct_unemployment']]
X = sm.add_constant(X)
y = features.life_expectancy

model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        life_expectancy   R-squared:                       0.490
Model:                            OLS   Adj. R-squared:                  0.490
Method:                 Least Squares   F-statistic:                     8997.
Date:                Tue, 10 Sep 2019   Prob (F-statistic):               0.00
Time:                        12:45:37   Log-Likelihood:            -1.6208e+05
No. Observations:               65656   AIC:                         3.242e+05
Df Residuals:                   65648   BIC:                         3.243e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               81.2365      0.122    665.628      0.000      80.997      81.476
pct_black           -0.0666      0.001    -56.960      0.000      -0.069      -0.064
pct_white           -0.0386      0.001    -36.707      0.000      -0.041      -0.037
pct_hispanic         0.0131      0.001     10.298      0.000       0.011       0.016
pct_less_than_hs    -0.0862      0.002    -48.979      0.000      -0.090      -0.083
pct_1_15_poverty    -0.0596      0.003    -21.738      0.000      -0.065      -0.054
income_10k           0.4825      0.006     83.217      0.000       0.471       0.494
pct_unemployment    -0.1490      0.004    -33.408      0.000      -0.158      -0.140
==============================================================================
Omnibus:                     2114.193   Durbin-Watson:                   1.520
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4788.035
Skew:                           0.183   Prob(JB):                         0.00
Kurtosis:                       4.271   Cond. No.                         790.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [68]:
import statsmodels.formula.api as smf

model = smf.ols(formula='life_expectancy ~ pct_black + pct_white + \
                        pct_hispanic + pct_less_than_hs + pct_1_15_poverty + \
                        income_10k + pct_unemployment', data=features)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        life_expectancy   R-squared:                       0.490
Model:                            OLS   Adj. R-squared:                  0.490
Method:                 Least Squares   F-statistic:                     8997.
Date:                Tue, 10 Sep 2019   Prob (F-statistic):               0.00
Time:                        13:29:46   Log-Likelihood:            -1.6208e+05
No. Observations:               65656   AIC:                         3.242e+05
Df Residuals:                   65648   BIC:                         3.243e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           81.2365      0.122    665.628      0.000      80.997      81.476
pct_black           -0.0666      0.001    -56.960      0.000      -0.069      -0.064
pct_white           -0.0386      0.001    -36.707      0.000      -0.041      -0.037
pct_hispanic         0.0131      0.001     10.298      0.000       0.011       0.016
pct_less_than_hs    -0.0862      0.002    -48.979      0.000      -0.090      -0.083
pct_1_15_poverty    -0.0596      0.003    -21.738      0.000      -0.065      -0.054
income_10k           0.4825      0.006     83.217      0.000       0.471       0.494
pct_unemployment    -0.1490      0.004    -33.408      0.000      -0.158      -0.140
==============================================================================
Omnibus:                     2114.193   Durbin-Watson:                   1.520
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4788.035
Skew:                           0.183   Prob(JB):                         0.00
Kurtosis:                       4.271   Cond. No.                         790.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Translate some of your coefficients into the form **"every X percentage point change in unemployment translates to a Y change in life expectancy."** Do this with numbers that are meaningful, and in a way that is easily understandable to your reader.

In [60]:
# A 1 percentage point increase in unemployment translates to a 0.15 year decrease in life expectancy

In [61]:
# A 10 percentage point increase in unemployment translates to a 1.5 year decrease in life expectancy

> Do your numbers seem off? Things too big, or too small? Make sure your percentages are percentage points between 0 and 100, not fractions between 0 and 1.